In [1]:
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import LSTM, Dense
from keras import Sequential
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

## data preprocessing/loading

In [23]:
# comment first line out after first use to prevent 
# giving the source website for the stocks too much traffic. 
stocks_df = ingest_stocks_to_df()
stock_list = stocks_df.company_name.unique()
# displays
display(stocks_df)
for stock in stock_list:
    print(stock, 'len', len(stocks_df[stocks_df.company_name == stock].index))
sns.lineplot(x=stocks_df.index, y='Adj Close',data=stocks_df, legend='auto', hue='company_name')
plt.show()
data_gen_train, data_gen_test = to_ts_data(stock_list[0], stocks_df) 

## Model Building, 
exploration of best in first round
- bi_stack_lstm_model(): 0.49 MSE test, 2.7211 train
- lstm_dense_model(): 0.4828 MSE test, 2.7904 train
- gru_model() 0.5256 MSE test, 2.4440 train
- stack_3_lstm_model 0.558 MSE test, 2.9509 train

# Pre-existing models

In [14]:
def gru_model(nodes=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    model = Sequential()
    model.add(GRU(nodes))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def stack_3_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(lstm_layer)
    model.add(lstm_layer)
    model.add(lstm_layer)
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def lstm_dense_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, dropout=dropout, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(Dense(32))
    model.add(Activation(relu))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
def bi_stack_lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, recurrent_dropout=recurrent_dropout, return_sequences=True)
    model = Sequential()
    model.add(Bidirectional(lstm_layer))
    model.add(Bidirectional(lstm_layer))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model

## Model eval
iterative predicting into the future; plotting. 

In [15]:
early_stop = EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)

In [16]:
model = gru_model()
model.fit((data_gen_train), epochs=300,callbacks=[early_stop])
model.evaluate(data_gen_test)

NameError: name 'Sequential' is not defined